# Project SoyHuCe

L'objectif est ici d'identifier une baleine à partir d'une photo de sa queue.

Il y a 4 250 noms de baleine différents inscrits dans les données d'entraînement, ainsi que 810 images qui sont étiquetées *new_whale*. Il y a en tout plus de 25 000 images.

> Notebook réalisé par Lancelot Godement pour l'entreprise SoyHuCe.

#### les imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
import os

On définit nos chemins d'environnement qui nous serviront pour tout le reste du projet.

In [ ]:
projF  = "../input/whale-categorization-playground/"
testF  = "test/"
trainF = "train/"

# Entraînement

On récupère les données dont on a besoin : les flags associés aux images d'entraînement.

In [ ]:
dataflag = pd.read_csv( projF + "train.csv" )
dataflag.head()

# 1re idée : l'extraction d'objet au premier plan

L'idée ici est d'isoler sur chaque image et dans la mesure du possible, l'objet au premier plan de celle-ci, c'est-à-dire la queue de l'animal. Dans le plan initial, nous souhaitions faire tourner l'algorithme d'apprentissage sur ces données de façon à se focaliser sur les données les plus importantes.

Cette idée a par la suite été abandonnée, car elle semble d'une complexité beaucoup trop élevé. Si cela ne pose pas de problème sur le petit nombre d'images utilisé pour l'exemple, dans un cas d'une base de données encore plus conséquente que celle utilisée dans le présent exemple, cela peut devenir une contrainte des plus bloquante.

Description des fonctions :
* load : charge l'image sous la forme d'une matrice d'une taille définie, puis reconvertie l'image dans un nouvel espace de colorimétrie.
* adaptive_hist : on améliore l'image en utilisant OpenCV CLAHE (Contrast Limited Adaptive Histogram Equalization). Pour éviter une augmentation trop forte du gain sur la totalité de l'image, on découpe cette dernière en grille avant d'appliquer l'égalisation d'histogramme sur chaque case.
* k_means : améliore la qualité de l'image en utilisant l'algorithme d'apprentissage en créant une version "compressée" de celle-ci.
* find_box : trace une boite autour de l'objet à cibler, via la technologie de OpenCV : [findContours](https://docs.opencv.org/master/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0).
* forgrd_ext : extrait l'objet au premier plan.

In [ ]:
def load(path, size=128):
    return cv2.cvtColor( cv2.resize( cv2.imread(path),(size,size) ), cv2.COLOR_BGR2RGB)

def adaptive_hist(img, clipLimit= 4.0):
    window    = cv2.createCLAHE( clipLimit = clipLimit, tileGridSize = (8, 8) )
    img_lab   = cv2.cvtColor( img, cv2.COLOR_BGR2Lab )
    ch1, ch2, ch3 = cv2.split( img_lab )
    img_l     = window.apply( ch1 )
    img_clahe = cv2.merge( (img_l, ch2, ch3) )
    return cv2.cvtColor( img_clahe, cv2.COLOR_Lab2BGR )

from sklearn.cluster import KMeans

def k_means(img, n_colors = 4):
    w, h, d  = original_shape = tuple(img.shape)
    img      = img/255.0
    image_array = np.reshape( img, (w * h, d) )
    kmeans   = KMeans( n_clusters = n_colors, random_state = 0 ).fit( image_array )
    labels   = kmeans.predict( image_array )
    codebook = kmeans.cluster_centers_
    d        = codebook.shape[1]
    image    = np.zeros((w, h, d))
    label_idx = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = codebook[labels[label_idx]]
            label_idx += 1
    return image

def find_box(edges): 
    co, hi  = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    con     = max(co, key=cv2.contourArea)
    conv_hull = cv2.convexHull(con)
    
    top     = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom  = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left    = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right   = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    return top, bottom, left, right

def forgrd_ext(img, rec):
    mask    = np.zeros(img.shape[:2], np.uint8)
    bgmodel = np.zeros((1, 65), np.float64)
    fgmodel = np.zeros((1, 65), np.float64)
    
    cv2.grabCut(img, mask, rec, bgmodel, fgmodel, 3, cv2.GC_INIT_WITH_RECT)
    
    mask2   = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')
    img     = img*mask2[:,:,np.newaxis]
    img[np.where( (img == [0,0,0]).all(axis = 2) )] = [255.0, 255.0, 255.0]
    return img

In [ ]:
def ext_frgd():
    f, ax = plt.subplots(5, 5, figsize=(40,30))
    for i in tqdm(range(25)):
        
        path   = os.path.join( projF + trainF, dataflag.Image[i])
        img_id = dataflag.Id[i]
        
        img    = load(path, 300)
        img    = adaptive_hist(img, clipLimit = 4.0)
        org    = img.copy()
        img    = k_means(img , n_colors= 10)
        
        img_gray = cv2.cvtColor(np.uint8(img*255), cv2.COLOR_RGB2GRAY)
        img_gray = cv2.medianBlur(img_gray,7)
        edges    = cv2.Canny(img_gray,100,200)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,15))
        edges  = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
        
        top,bottom,left,right = find_box(edges)
        rec = (left[0], top[1], right[0]-left[0], bottom[1]-top[1])
        forground_img = forgrd_ext(org, rec)
        
        ax[i//5][i%5].imshow(forground_img, aspect='auto')
        ax[i//5][i%5].set_title(img_id)
        ax[i//5][i%5].set_xticks([]); ax[i//5][i%5].set_yticks([])
    plt.show()

In [ ]:
ext_frgd()

---


# 2me idée : faire un apprentissage plus classique

Plus direct de conception, cette idée se base sur le projet [CNN with Keras for Humpback Whale ID](https://www.kaggle.com/anezka/cnn-with-keras-for-humpback-whale-id) dont nous avons tout de fois optimisé le code, forçant ainsi sa compréhension dans son intégralité. Il s'agit ici d'associer les matrices d'images à l'*Id* qui les décrits, puis d'utiliser l'apprentissage artificiel : un réseau neuronal convolutif (CNN), de façon à être capable de prédire à partir d'une image inconnue le nom ou les noms potentiels d'une baleine.

#### Import

In [ ]:
%matplotlib inline

import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

## Préparation d'image

On définit les images de façon à ce qu'elles soient encodées sous la forme de matrices interprétables par notre système.

In [ ]:
def prepareImages(pathD, data, m, dataset):
    
    X_train = np.zeros((m, 100, 100, 3))
    
    for i in tqdm(range(len(data['Image']))) :
        img = image.load_img(pathD + data['Image'][i], target_size = (100, 100, 3)) # charge les images sous la forme d'une matrice de 100 * 100 * 3
        x   = image.img_to_array(img) #                                               transforme l'image en array
        x   = preprocess_input(x)     #                                               traite les données sous la forme d'un tableau numpy
        X_train[i] = x
        
    X_train /= 255
    return X_train

In [ ]:
X = prepareImages(projF + trainF, dataflag, dataflag.shape[0], "train")

## Préparation des Labels
On encode les labels de façon à ce qu'ils puissent être représentés sous une forme booléenne.
Ainsi, les données sont représentées sous la forme :
> 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

Chaque colonne représentant un *Id* particulier.

In [ ]:
def prepare_labels(y):
    
    values          = np.array(y)
    label_encoder   = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values) #              transforme les valeurs en entier
    print(integer_encoded)
    
    onehot_encoder  = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1) # retourne la matrice
    onehot_encoded  = onehot_encoder.fit_transform(integer_encoded)    # met les données sous forme booléenne
    print(onehot_encoded)
    
    y               = onehot_encoded
    print(y.shape)
    return y, label_encoder

In [ ]:
y, label_encoder = prepare_labels(dataflag['Id'])

In [ ]:
print("X-train shape :", X.shape)
print("y-train shape :", y.shape)

## On définit le modèle d'apprentissage

L'apprentissage peut prendre un temps assez conséquent. Il est donc vivement conseillé d'utiliser un GPU pour cela.

In [ ]:
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3))) # Convolution ajoutant un biais aux données

model.add(BatchNormalization(axis = 3, name = 'bn0')) #                                        Normalise les inputs tout en limitant les pertes d'informations
model.add(Activation('relu')) #                                                                Applique la fonction d'activation basée sur la fonction Relu

model.add(MaxPooling2D((2, 2), name='max_pool')) #                                             Couche de mise en commun maximum
model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool')) #                                         Couche de mise en commun moyenne

model.add(Flatten()) #                                                                         Applatit l'entrée
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.8)) #                                                                      Met de côté les valeurs les moins représenté pour éviter le surajustement
model.add(Dense(4251, activation='softmax', name='sm'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()
print(model.output_shape)

In [ ]:
history = model.fit(X, y, epochs=100, batch_size=100, verbose=1)
gc.collect()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

# On passe à la pratique sur les données de test
Comme expliqué dans le projet *CNN with Keras for Humpback Whale ID*, nous divisons les données en plusieurs parties de façon à ne pas être bloqué par la taille de celle-ci.

In [ ]:
test      = os.listdir( projF + testF )
col       = ['Image']
testData  = pd.DataFrame(test,             columns = col)
testData1 = pd.DataFrame(test[:3900],      columns = col)
testData2 = pd.DataFrame(test[3900:7800],  columns = col)
testData3 = pd.DataFrame(test[7800:11700], columns = col)
testData4 = pd.DataFrame(test[11700:],     columns = col)

On prédit puis concatène toutes les prédictions.

In [ ]:
testAllData = [testData1, testData2, testData3, testData4]
prediction = []
for data in testAllData :
    X = prepareImages(projF + testF, data, data.shape[0], "test")
    prediction += [model.predict(np.array(X), verbose = 1)]
    gc.collect()

In [ ]:
predictions = np.concatenate(prediction, axis=0)
gc.collect()
print(predictions.shape)
print(predictions)

In [ ]:
copy_predB = np.copy(predictions)
idx       = []
for i in tqdm(range(5)) :
    idx += [np.argmax(copy_predB, axis=1)]
    copy_predB[:,idx[i]] = 0

On retransforme les *Id* de leur forme numérique à leur forme textuelle.
On en profite pour appliquer un seuil de précision de façon à ne garder que les prédictions les plus probables.

In [ ]:
results   = []
threshold = 0.05 # seuil de précision

for i in tqdm(range(0, predictions.shape[0])):
    each = []
    tags = []
    for j in range(len(idx)):
        each += [np.zeros((4251, 1))]
    if((predictions[i, idx[4][i]] > threshold)):
        tags = []
        for j in range(len(idx)):
            each[j][idx[j][i]] = 1
            tags += [label_encoder.inverse_transform([np.argmax(each[j])])[0]]
            
    elif((predictions[i, idx[3][i]] > threshold)):
        print(predictions[i, idx[3][i]])
        for j in range(len(idx)-1):
            each[j][idx[j][i]] = 1
        tags = []
        for j in range(len(idx)-1):
            tags += [label_encoder.inverse_transform([np.argmax(each[j])])[0]]
            
    elif((predictions[i, idx[2][i]] > threshold)):
        for j in range(len(idx)-2):
            each[j][idx[j][i]] = 1
        tags = []
        for j in range(len(idx)-2):
            tags += [label_encoder.inverse_transform([np.argmax(each[j])])[0]]
            
    elif((predictions[i, idx[1][i]] > threshold)):
        for j in range(len(idx)-3):
            each[j][idx[j][i]] = 1
        tags = []
        for j in range(len(idx)-3):
            tags += [label_encoder.inverse_transform([np.argmax(each[j])])[0]]
            
    else:
        each[0][idx[0][i]] = 1
        tags = [label_encoder.inverse_transform([np.argmax(each[0])])[0]]
    results += [tags]

In [ ]:
results

### Dernière étape : on enregistre nos résultats

In [ ]:
import csv
myfile = open('output.csv','w')
column = ['Image', 'Id']

wrtr   = csv.writer(myfile, delimiter=',')
wrtr.writerow(column)

for i in tqdm(range(testData.shape[0])):
    pred = ""
    for j in range(len(results[i])):
        if j != 0:
            pred += " "
        pred += results[i][j]
            
    result = [testData['Image'][i], pred]
    wrtr.writerow(result)
    
myfile.close()

# Conclusion

L'objectif de cet exercice était de parvenir à prédire le nom d'une baleine à partir de sa queue. De façon objective, nous parvenons à un résultat assez satisfaisant.

D'un point de vue méthodologique, si je n'ai pas produit l'entièreté du code par moi-même, j'ai tenu à le comprendre et à l'optimiser de façon à ce qu'il soit le plus possible à appréhender, et cela, même pour quelqu'un qui n'est pas spécialiste du domaine.

N'ayant pas encore beaucoup d'expérience dans le domaine, ainsi qu'un nombre d'applications réelles très limité, je souhaiterais apprendre un maximum de chose auprès des professionnels que vous êtes.

> GODEMENT Lancelot